In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import platform
if platform.system() == "Darwin":          # MAC인 경우
    plt.rc('font', family ="AppleGothic")
else:
    plt.rc('font',family='Malgun Gothic')

#멧플로립에서 음수를 표시
plt.rc('axes',unicode_minus=False)

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')

In [4]:
pd.read_csv("./fin_raw_data.csv.csv", encoding='cp949')

FileNotFoundError: [Errno 2] No such file or directory: './fin_raw_data.csv.csv'

In [3]:
# 파일 불러오기 

df1 = pd.read_csv("./fin_raw_data.csv.csv", encoding='cp949')
# df1.info()
df1 = df1.replace(',', '')
df1['달러원종가'] = pd.to_numeric(df1['달러원종가'], errors='ignore')
df1['달러원시가'] = pd.to_numeric(df1['달러원시가'], errors='ignore')
df1['달러인덱스'] = pd.to_numeric(df1['달러인덱스'], errors='ignore')

FileNotFoundError: [Errno 2] No such file or directory: './raw_data2.csv'

In [ ]:
# 전처리 하기 
# df2 =pd.merge(df1,df,how='inner',on='날짜')
df2=pd.read_csv("./raw_data_joo.csv")
df2['달러원종가'] = df2['달러원종가'].replace(',', '')
df2['달러원시가'] = df2['달러원시가'].replace(',', '')

# df2.info()

# df2['달러원시가']=df2['달러원시가'].astype('float')
df2['콜금리']=df2['콜금리'].astype('float')

# df2['달러인덱스']=df2['달러인덱스'].astype('float')
# df2['BIR']=df2['BIR'].astype('float')

for i in df2.index:
    if df2.loc[i,'달러인덱스'] == '.':
        df2.loc[i,'달러인덱스'] = np.nan

for i in df2.index:
    if df2.loc[i,'BIR'] == '.':
        df2.loc[i,'BIR'] = np.nan

df2.isna().sum()

In [ ]:
# df2['콜금리']=df2['콜금리'].astype('float')

df2['달러인덱스']=df2['달러인덱스'].astype('float')
df2['달러인덱스'] = df2['달러인덱스'].fillna(method='bfill')

df2['BIR']=df2['BIR'].astype('float')
df2['BIR'] = df2['BIR'].fillna(method='bfill')
df2['코스피종가'] = df2['코스피종가'].fillna(method='bfill')

#df2['달러원시가']=df2['달러원시가'].astype('float')
# df2['달러원종가']=df2['달러원종가'].astype('float')

df2.info()

In [ ]:
df2['달러원종가']=df2['달러원종가'].str.replace(',','')
df2['달러원종가']=df2['달러원종가'].astype('float')
df2.info()

In [ ]:
# 변수에 따라서 다중공정성없애기 
# 다중공정성으로 CD 91물, 달러엔 제외 
corr_data = df2[[ 'WTI가', '달러위안화',  '달러인덱스', '유로달러',
       '유로인덱스', '콜금리', '코스피종가', 'BIR']].corr()
plt.figure(figsize=(10, 8))

mask = np.triu(np.ones_like(corr_data, dtype=bool))

sns.heatmap(corr_data, annot=True, cmap='coolwarm', linewidths=.5, fmt='.2f', mask=mask)



In [ ]:
# 달러원 종가와 상관관계 높은것이 피쳐셀렉에 논리적인가오ㅛ? 

corr_data = df2[['달러원종가', 'CD91', 'WTI가', '달러위안화', '달러엔', '달러인덱스', '유로달러',
       '유로인덱스', '콜금리', '코스피종가', 'BIR']]

np.abs(corr_data.corr()['달러원종가']).sort_values(ascending=False)




In [ ]:
# 변수 6개만 선택하기 + 스켈링하기 + 단위근 조정하기 (Minmax scaler ) 
corr_data = df2[[ '달러위안화',   '유로달러','달러인덱스',
       '유로인덱스','코스피종가' ]]

plt.figure(figsize=(12,8))
sns.heatmap(corr_data.corr(), annot=True)

# from sklearn.preprocessing import MinMaxScaler
# minmax_scaler = MinMaxScaler()
# minmax_scaler.fit(corr_data)
# minmax_scaled_data = minmax_scaler.transform(corr_data)
# minmax_scaled_df = pd.DataFrame(data=minmax_scaled_data, columns=corr_data.columns )
# minmax_scaled_df.describe


In [ ]:
from sklearn.preprocessing import StandardScaler

Standard_Scaler = StandardScaler()
Standard_Scaler.fit(corr_data)
Standard_Scaled_data = Standard_Scaler.transform(corr_data)
Standard_Scaled_df = pd.DataFrame(data=Standard_Scaled_data, columns=corr_data.columns)
Standard_Scaled_df.describe()

# weight: [ 1.03691204  0.12470911  0.13843903 -0.27187524 -0.31934538 -1.24699102
#  0.67207078 -0.88829846 -0.24143534 -0.11441595]
# bias: 0.0013727852680168876
# 선형회귀 모델 R2: 0.86

In [ ]:
from sklearn.model_selection import train_test_split
y= df2['달러원종가']
X=Standard_Scaled_df


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

print('weight:', lr_model.coef_)
print('bias:', lr_model.intercept_)
# 이건 뭐에 대한 score 인가?? 
score =lr_model.score(X_test, y_test)

print('선형회귀 모델 R2: {0:.2f}'.format(score))
score

In [ ]:
# Cofficient 값을 버리고, 부호만 살리는건 어때요??
coef=pd.Series(data=np.round(lr_model.coef_,5), index=X.columns)
coef

In [ ]:
# 이 시각화는 왜 이모냥인겁니까?? 
coef_sort=coef.sort_values(ascending=False)
sns.barplot(x=coef_sort.values, y=coef_sort.index)
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

def printRegressorResult(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    rmse= np.sqrt(mse)
    r2=r2_score(y_test, y_pred)
    print('MSE:{0:3f}, RMSE:{1:.3f}, r2:{2:.3f}'.format(mse,rmse,r2))

y_pred=lr_model.predict(X_test)
printRegressorResult(y_test,y_pred)

In [ ]:
result=pd.DataFrame({'y':y_test.values,'y_pred':y_pred, 'diff':np.abs(y_test.values-y_pred)})
result.sort_values(by=['diff'], ascending=False)

In [ ]:
# 에측결과 시각화, 왜 에러가 날까요??
y_pred = lr_model.predict(X_test)
lr_df = pd.DataFrame(data={'Y':y_test, 'Y_pred(lr)':y_pred,'diff(lr)':np.round(np.abs(y_test-y_pred),2)})
sns.lineplot(data=lr_df)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)

score=tree_model.score(X_test,y_test)
print('의사결정회귀 R2: {0:.2f}'.format(score))

In [ ]:
y_pred = tree_model.predict(X_test)

tree_df = pd.DataFrame(data = {'Y': y_test, 'Y_pred(tree)':y_pred, 'diff(tree)':np.round(np.abs(y_test-y_pred),2)} )
tree_df.sort_values(by=['diff(tree)'],ascending=False).head(10)

In [ ]:
# 이거 PPT 에 사용하면 좋겠슴당!! 
sns.lineplot(data=tree_df)

In [ ]:
!pip install lightgbm

In [ ]:
# 이건 왜 다 같은 값이 나올까요?? 
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

forest_model = RandomForestRegressor(n_estimators=100, random_state=42)
lgb_model = LGBMRegressor(n_estimators=100, random_state=42)
xgb_model = XGBRegressor(n_estimators=100, random_state=42)

model_list = [forest_model, lgb_model, xgb_model]

for model in model_list:
    model.fit(X_train, y_train)
    score=model.score(X_test, y_test)
    model_name = model.__class__,__name__
    print('{0} 모델 R2: {1:.2f}'.format(model_name,score))


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

def printRegressorResult(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    rmse=np.sqrt(mse)
    r2=r2_score(y_test, y_pred)
    print('MSE:{0:.3f}, RMSE:{1,.3f}, r2:{2,.3f}'.format(mse, rmse, r2))

In [ ]:
# 이건 왜 에러가 날까요?
model_list=[lr_model, tree_model, forest_model, lgb_model, xgb_model]

for model in model_list:
    y_preds = model.predict(X_test)
    print('{0} 모델'.format(model.__class__.__name__))
    printRegressorResult(y_test, y_preds)

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

X_selected=SelectKBest(score_func= f_regression, k=5)
X_selected.fit_transform(X,y)
features = X.columns[X_selected.get_support()]
print('features={}'.format(features))

corr_features = corr_data[['달러위안화', '유로달러', '달러인덱스', '유로인덱스','코스피종가']]

In [ ]:
corr_features

In [ ]:
X_selected=corr_features
X_train, X_test, y_train, y_test= train_test_split(X_selected,y,test_size=0.3, random_state=42)
lr_model2=LinearRegression()
lr_model2.fit(X_train, y_train)

y_pred = lr_model2.predict(X_test)
printRegressorResult(y_test,y_pred)

# 왜 점수가 요렇게 나오까요? 

In [ ]:
# polynomialFeatures() 사용하여 2차원 다항회귀모델

from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2)
X_poly = poly_features.fit_transform(X_selected)
X_train, X_test, y_train, y_test=train_test_split(X_poly, y, test_size=0.3, random_state=42)

poly_model=LinearRegression()
poly_model.fit(X_train, y_train)

y_pred = poly_model.predict(X_test)
printRegressorResult(y_test, y_pred)

In [ ]:
# 데이터 전처리 

from sklearn.preprocessing import StandardScaler
X_selected_scaled = StandardScaler().fit_transform(X_selected)
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

lr_model3=LinearRegression()
lr_model3.fit(X_train, y_train)

y_pred = lr_model3.predict(X_test)
printRegressorResult(y_test,y_pred)

In [ ]:
# 데이터 분포 확인- 히스토그램 
nrows=1
ncols=4

fig,axs=plt.subplots(nrows, ncols)
fig.set_size_inches(20,4)

sns.histplot(x=X_selected.columns[0], data=X_selected, kde=True, bins=30, ax=axs[0])
sns.histplot(x=X_selected.columns[1], data=X_selected, kde=True, bins=30, ax=axs[1])
sns.histplot(x=X_selected.columns[2], data=X_selected, kde=True, bins=30, ax=axs[2])
sns.histplot(x=X_selected.columns[3], data=X_selected, kde=True, bins=30, ax=axs[3])


sns.histplot(x=y, data=y, kde=True, bins=30, ax=axs[3])

print(X_selected.skew())
print('\n달러원종가:{0,.2f}'.format(y.skew()))

In [ ]:
# 로그연산 유로달러 해보기 
X_selected['유로달러']=np.log1p(X_selected['유로달러'])
y=np.log1p(y)

# 데이터분포확인 - 히스토그램 
nrows=1
ncols=4

fig, axs=plt.subplots(nrows, ncols)
fig.set_size_inches(20,4)

sns.histplot(x=X_selected.columns[0], data=X_selected, kde=True, bins=30, ax=axs[0])
sns.histplot(x=X_selected.columns[1], data=X_selected, kde=True, bins=30, ax=axs[1])
sns.histplot(x=X_selected.columns[2], data=X_selected, kde=True, bins=30, ax=axs[2])
sns.histplot(x=y, data=y, kde=True, bins=30, ax=axs[3])

print(X_selected.skew())
print('\n달러원종가:{0:.2f}'.format(y.skew()))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_selected,y,test_size=0.3, random_state=42)

lr_model4= LinearRegression()
lr_model4.fit(X_train, y_train)

y_pred= lr_model4.predict(X_test)
mse=mean_squared_error(np.expm1(y_test), np.expm1(y_pred))
rmse=np.sqrt(mse)
r2=r2_score(np.expm1(y_test), np.expm1(y_pred))
print('MSE : {0:.3f}, RMSE : {1:.3f}, r2 : {2:.3f}'.format(mse, rmse, r2))

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(lr_model, X, y, scoring='neg_mean_squared_error', cv=3, return_train_score=True, return_estimator=True)
print('Scores', scores)

mse=(-1*scores['train_score'])
print('MSE:', mse)

rmse=np.sqrt(-1*scores['train_score'])
print('RMSE:', rmse)

print('RMSE 평균:{0:.3f}'.format(np.mean(rmse)))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor(n_estimators=100, random_state=42)
forest_model.fit(X_train, y_train)

In [ ]:
!pip install lightgbm

In [ ]:
from lightgbm import LGBMRegressor

lgb_model=LGBMRegressor(n_estimators=100, random_state=42)
lgb_model.fit(X_train, y_train)

In [ ]:
model_list=[lr_model, forest_model, lgb_model]

for model in model_list:
    model.fit(X_train, y_train)
    y_preds=model.predict(X_test)
    mse=mean_squared_error(y_test, y_preds)
    rmse = np.sqrt(mse)
    r2=r2_score(y_test, y_preds)
    model_name=model.__class__.__name__
    print('{0} MSE:{1:.3f}, RMSE:{2:.3f}, r2:{3:.3f}'.format(model_name, mse, rmse,r2))